# 데이터 전처리 및 저장

In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import openpyxl
import matplotlib.pyplot as plt
from collections import defaultdict
import time
import pickle
import logging
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise import accuracy

In [2]:
# 데이터 불러오기 
data_path = '/Users/kyoungmin/recommendSystem/data'
rating_raw = pd.read_excel(os.path.join(data_path, 'rating.xlsx'))

In [3]:
rating_raw.head(5)

,타임스탬프,"1. [1번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요.,다른 음식점을 추가로 입력하시겠습니까?,"1. [2번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..1,다른 음식점을 추가로 입력하시겠습니까?.1,"1. [3번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..2,다른 음식점을 추가로 입력하시겠습니까?.2,"1. [4번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..3,다른 음식점을 추가로 입력하시겠습니까?.3,"1. [5번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..4,2. 선호하는 음식 카테고리를 모두 선택해주세요 (복수 선택 가능),3. 평소 한 끼 식비로 사용하는 금액은 얼마인가요?\n( 숫자만 입력해 주세요. 예: 8000 ),"4. 개인정보 수집 및 활용 동의 (기프티콘 발송용)\n※ 수집된 전화번호는 당첨자에게 기프티콘 발송을 위한 연락 용도 외에는 일절 사용되지 않으며, 설문 종료 즉시 안전하게 폐기됩니다.",5. 기프티콘 발송용 연락처 입력\n(예: 010-1234-1234)
0,2025-05-19 12:41:40.600,동대닭한마리,5,네,김치만선생,5.0,네,오이드킨,4.0,네,희희카츠,5.0,네,짬뽕야,3.0,"한식, 일식",12000,위 내용을 읽고 동의합니다.,010-6637-6782
1,2025-05-19 13:40:06.994,홍탁집,4,네,짬뽕야,5.0,네,김치만선생,5.0,네,봉추찜닭,5.0,네,필동족발,4.0,"한식, 양식, 아시안, 일식",12000,위 내용을 읽고 동의합니다.,010-4474-5864
2,2025-05-19 13:40:22.434,김치만,4,네,손문,4.0,네,아시아,5.0,네,소담집,4.0,네,산타돈부리,3.0,"한식, 중식, 양식, 아시안, 멕시칸, 일식",13000,위 내용을 읽고 동의합니다.,010-3693-9758
3,2025-05-19 13:46:00.776,닭한마리,5,네,한국의집,5.0,네,손문,4.0,네,하얀집,3.0,네,홍탁집,4.0,"한식, 양식, 일식",12000,위 내용을 읽고 동의합니다.,01073208549
4,2025-05-19 13:47:21.938,시오,4,네,필동 부대찌개,3.0,네,하얀집,2.0,네,닭한마리,5.0,네,파스타마켓,4.0,"한식, 양식, 분식",20000,위 내용을 읽고 동의합니다.,010-8920-6870


In [4]:
#필요 없는 열 삭제
rating = rating_raw.drop(rating_raw.columns[[0,3,6,9,12,17,18]], axis=1)

In [5]:
rating.head()

,"1. [1번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요.,"1. [2번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..1,"1. [3번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..2,"1. [4번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..3,"1. [5번째] 충무로, 명동, 을지로 3가 인근에서 실제로 방문해 본 음식점 이름을 작성해주세요. \n(‘가게 이름만’ 간단히 적어주세요. (ex. 한끼천국 / 한끼치킨))",작성하신 음식점의 만족도를 평가해주세요..4,2. 선호하는 음식 카테고리를 모두 선택해주세요 (복수 선택 가능),3. 평소 한 끼 식비로 사용하는 금액은 얼마인가요?\n( 숫자만 입력해 주세요. 예: 8000 )
0,동대닭한마리,5,김치만선생,5.0,오이드킨,4.0,희희카츠,5.0,짬뽕야,3.0,"한식, 일식",12000
1,홍탁집,4,짬뽕야,5.0,김치만선생,5.0,봉추찜닭,5.0,필동족발,4.0,"한식, 양식, 아시안, 일식",12000
2,김치만,4,손문,4.0,아시아,5.0,소담집,4.0,산타돈부리,3.0,"한식, 중식, 양식, 아시안, 멕시칸, 일식",13000
3,닭한마리,5,한국의집,5.0,손문,4.0,하얀집,3.0,홍탁집,4.0,"한식, 양식, 일식",12000
4,시오,4,필동 부대찌개,3.0,하얀집,2.0,닭한마리,5.0,파스타마켓,4.0,"한식, 양식, 분식",20000


In [6]:
# 식당 이름 컬럼 (0, 2, 4, 6, 8번째)
restaurant_cols = [0, 2, 4, 6, 8]
# 평점 컬럼 (1, 3, 5, 7, 9번째)  
rating_cols = [1, 3, 5, 7, 9]

# 식당 이름과 평점 데이터만 추출
restaurants = rating.iloc[:, restaurant_cols]
ratings = rating.iloc[:, rating_cols]

In [7]:
# 식당 이름 컬럼 (0, 2, 4, 6, 8번째)
restaurant_cols = [0, 2, 4, 6, 8]
# 평점 컬럼 (1, 3, 5, 7, 9번째)  
rating_cols = [1, 3, 5, 7, 9]

# 식당 이름과 평점 데이터만 추출
restaurants = rating.iloc[:, restaurant_cols]
ratings = rating.iloc[:, rating_cols]

In [8]:
import pandas as pd

# 사용자 ID 생성 (행 인덱스 사용)
user_data = []

for user_id in range(len(rating)):
    for i in range(5):  # 5개 식당
        restaurant_name = restaurants.iloc[user_id, i]
        rating_score = ratings.iloc[user_id, i]
        
        # 식당 이름과 평점이 모두 있는 경우만 추가
        if pd.notna(restaurant_name) and pd.notna(rating_score):
            user_data.append({
                'user_id': user_id,
                'restaurant_name': restaurant_name,
                'rating': float(rating_score)
            })

# DataFrame 생성
rating_long = pd.DataFrame(user_data)

In [9]:
# 식당 이름 정규화 (공백만 제거)
rating_long['restaurant_name'] = rating_long['restaurant_name'].str.strip()

# 유니크한 식당 목록과 ID 매핑
unique_restaurants = rating_long['restaurant_name'].unique()
# 매핑 딕셔너리 (이름 → ID)
restaurant_mapping = {name: idx for idx, name in enumerate(unique_restaurants)}
# 역매핑 딕셔너리 (ID → 이름) 
restaurant_reverse_mapping = {idx: name for name, idx in restaurant_mapping.items()}

# 식당 ID 추가
rating_long['restaurant_id'] = rating_long['restaurant_name'].map(restaurant_mapping)
# 필요한 컬럼만 남기고 정리
svd_data = rating_long[['user_id', 'restaurant_id', 'rating']].copy()
# 컬럼명 변경 
svd_data.columns = ['userId', 'restaurantId', 'rating']

# 데이터 확인
print("=== SVD++ 입력용 최종 데이터 ===")
print(f"데이터 형태: {svd_data.shape}")
print(f"컬럼: {svd_data.columns.tolist()}")
print("\n샘플 데이터:")
print(svd_data.head(10))

=== SVD++ 입력용 최종 데이터 ===
데이터 형태: (148, 3)
컬럼: ['userId', 'restaurantId', 'rating']

샘플 데이터:
   userId  restaurantId  rating
0       0             0     5.0
1       0             1     5.0
2       0             2     4.0
3       0             3     5.0
4       0             4     3.0
5       1             5     4.0
6       1             4     5.0
7       1             1     5.0
8       1             6     5.0
9       1             7     4.0


In [10]:
from surprise import Dataset, Reader

# SVD++ 입력용 데이터셋 생성
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(svd_data, reader)

print("✅ SVD++ 입력용 데이터셋 생성 완료!")

✅ SVD++ 입력용 데이터셋 생성 완료!


In [11]:
#매핑 딕셔너리 저장
import pickle

# 매핑 딕셔너리들 저장
mappings = {
    'restaurant_to_id': restaurant_mapping,
    'id_to_restaurant': restaurant_reverse_mapping
}

# 파일로 저장
with open('restaurant_mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)

print("매핑 딕셔너리가 'restaurant_mappings.pkl' 파일로 저장되었습니다.")

매핑 딕셔너리가 'restaurant_mappings.pkl' 파일로 저장되었습니다.


# 모델 학습 및 평가

## 그리드 서치

In [15]:
from surprise import SVDpp
from surprise.model_selection import train_test_split

# 바로 학습 시작
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)
model = SVDpp()
model.fit(trainset)

In [16]:
param_grid = {
                    'n_factors': [50, 100, 150],      # 잠재 요인 수
                    'n_epochs': [20, 30, 40],         # 학습 반복 횟수
                    'lr_all': [0.005, 0.01, 0.02],   # 학습률
                    'reg_all': [0.02, 0.05, 0.1]     # 정규화 파라미터
                }

In [23]:

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# GridSearchCV 실행
start_time = time.time()
gs = GridSearchCV(
    SVDpp, 
    param_grid, 
    measures=['rmse', 'mae'],
    cv=5,
    n_jobs=-1,  # 병렬 처리
    joblib_verbose=1
)

gs.fit(surprise_data)

end_time = time.time()
logger.info(f"튜닝 완료! 소요 시간: {end_time - start_time:.2f}초")
# 최적 파라미터 저장
best_params = gs.best_params['rmse']
cv_results = gs.cv_results

logger.info(f"최적 파라미터 (RMSE 기준): {best_params}")
logger.info(f"최적 RMSE: {gs.best_score['rmse']:.4f}")
logger.info(f"최적 MAE: {gs.best_score['mae']:.4f}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:    0.2s finished
INFO:__main__:튜닝 완료! 소요 시간: 0.16초
INFO:__main__:최적 파라미터 (RMSE 기준): {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.02, 'reg_all': 0.02}
INFO:__main__:최적 RMSE: 0.7691
INFO:__main__:최적 MAE: 0.6356


In [44]:
# 최적 파라미터의 CV 결과 추출
results_df = pd.DataFrame(cv_results)

# 최적 파라미터 조합 찾기
best_index = results_df['mean_test_rmse'].idxmin()
best_row = results_df.loc[best_index]

# 각 fold별 성능 추출
fold_columns = [col for col in results_df.columns if col.startswith('split') and 'test_rmse' in col]
fold_rmse_scores = [best_row[col] for col in fold_columns]

fold_mae_columns = [col for col in results_df.columns if col.startswith('split') and 'test_mae' in col]
fold_mae_scores = [best_row[col] for col in fold_mae_columns]

# 통계 계산
rmse_mean = np.mean(fold_rmse_scores)
rmse_std = np.std(fold_rmse_scores)
mae_mean = np.mean(fold_mae_scores)
mae_std = np.std(fold_mae_scores)

logger.info(f"RMSE - 평균: {rmse_mean:.4f}, 표준편차: {rmse_std:.4f}")
logger.info(f"MAE - 평균: {mae_mean:.4f}, 표준편차: {mae_std:.4f}")
logger.info(f"RMSE 범위: {np.min(fold_rmse_scores):.4f} ~ {np.max(fold_rmse_scores):.4f}")
logger.info(f"MAE 범위: {np.min(fold_mae_scores):.4f} ~ {np.max(fold_mae_scores):.4f}")

# 과적합 여부 판단
cv_stability = rmse_std / rmse_mean  # 변동 계수
logger.info(f"CV 안정성 (낮을수록 좋음): {cv_stability:.4f}")

if cv_stability > 0.1:
    logger.warning("⚠️ CV 결과 변동이 큼 - 과적합 의심")
else:
    logger.info("✅ CV 결과 안정적")

INFO:__main__:RMSE - 평균: 0.7691, 표준편차: 0.0943
INFO:__main__:MAE - 평균: 0.6356, 표준편차: 0.0775
INFO:__main__:RMSE 범위: 0.6435 ~ 0.8647
INFO:__main__:MAE 범위: 0.5348 ~ 0.7175
INFO:__main__:CV 안정성 (낮을수록 좋음): 0.1226


## 과적합 이슈 해결
1. 앙상블 기법
2. 파라미터 둔화
3. 훈련 데이터를 서브 샘플링

In [54]:
random_state =42
sampling_ratio=0.3

import random
random.seed(random_state)

# 모든 평점 데이터 추출
all_ratings = [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), r) 
                for u, i, r in trainset.all_ratings()]

# 지정된 비율만큼 샘플링
sample_size = int(len(all_ratings) * sampling_ratio)
subsampled_ratings = random.sample(all_ratings, sample_size)

logger.info(f"📊 데이터 서브샘플링: {len(all_ratings)} → {len(subsampled_ratings)} ({sampling_ratio*100:.0f}%)")

INFO:__main__:📊 데이터 서브샘플링: 119 → 35 (30%)


In [52]:
# 최적 파라미터의 CV 결과 추출
results_df = pd.DataFrame(cv_results)

# 최적 파라미터 조합 찾기
best_index = results_df['mean_test_rmse'].idxmin()
best_row = results_df.loc[best_index]

# 각 fold별 성능 추출
fold_columns = [col for col in results_df.columns if col.startswith('split') and 'test_rmse' in col]
fold_rmse_scores = [best_row[col] for col in fold_columns]

fold_mae_columns = [col for col in results_df.columns if col.startswith('split') and 'test_mae' in col]
fold_mae_scores = [best_row[col] for col in fold_mae_columns]

# 통계 계산
rmse_mean = np.mean(fold_rmse_scores)
rmse_std = np.std(fold_rmse_scores)
mae_mean = np.mean(fold_mae_scores)
mae_std = np.std(fold_mae_scores)
                
logger.info(f"RMSE - 평균: {rmse_mean:.4f}, 표준편차: {rmse_std:.4f}")
logger.info(f"MAE - 평균: {mae_mean:.4f}, 표준편차: {mae_std:.4f}")
logger.info(f"RMSE 범위: {np.min(fold_rmse_scores):.4f} ~ {np.max(fold_rmse_scores):.4f}")
logger.info(f"MAE 범위: {np.min(fold_mae_scores):.4f} ~ {np.max(fold_mae_scores):.4f}")

# 과적합 여부 판단
cv_stability = rmse_std / rmse_mean  # 변동 계수
logger.info(f"CV 안정성 (낮을수록 좋음): {cv_stability:.4f}")

if cv_stability > 0.1:
    logger.warning("⚠️ CV 결과 변동이 큼 - 과적합 의심")
else:
    logger.info("✅ CV 결과 안정적")

# 추가적인 교차 검증 및 과적합 검사 (수동 구현)
train_rmse = None
test_rmse = None
overfitting_gap = None

from surprise.model_selection import KFold

# 최적 파라미터로 모델 생성
model = SVDpp(n_factors= 50, n_epochs= 30, lr_all=0.01, reg_all= 0.02)

# 수동 교차 검증으로 훈련/검증 성능 비교
kf = KFold(n_splits=5, random_state=42)
train_rmse_scores = []
test_rmse_scores = []

for trainset, testset in kf.split(surprise_data):
    # 모델 학습
    model.fit(trainset)
    
    # 훈련 세트 예측 (샘플링해서 계산)
    train_predictions = []
    # 훈련 세트에서 100개 샘플만 추출해서 계산 (메모리 절약)
    sample_size = min(100, trainset.n_ratings)
    import random
    
    train_sample = []
    all_train_ratings = [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), r) 
                        for u, i, r in trainset.all_ratings()]
    train_sample = random.sample(all_train_ratings, sample_size)
    
    for uid, iid, true_r in train_sample:
        pred = model.predict(uid, iid)
        train_predictions.append((true_r, pred.est))
    
    # 훈련 RMSE 계산
    train_rmse_fold = np.sqrt(np.mean([(true - pred)**2 for true, pred in train_predictions]))
    train_rmse_scores.append(train_rmse_fold)
    
    # 테스트 세트 예측
    test_predictions = model.test(testset)
    test_rmse_fold = accuracy.rmse(test_predictions, verbose=False)
    test_rmse_scores.append(test_rmse_fold)

# 평균 성능 계산
train_rmse = np.mean(train_rmse_scores)
test_rmse = np.mean(test_rmse_scores)
overfitting_gap = test_rmse - train_rmse

logger.info(f"훈련 RMSE (샘플 기반): {train_rmse:.4f}")
logger.info(f"검증 RMSE: {test_rmse:.4f}")
logger.info(f"과적합 차이: {overfitting_gap:.4f}")

# 과적합 판단
if overfitting_gap > 0.05:  # 임계값
    logger.warning(f"⚠️ 과적합 감지! Train-Test 차이: {overfitting_gap:.4f}")
elif overfitting_gap > 0.02:
    logger.info(f"⚠️ 약간의 과적합. Train-Test 차이: {overfitting_gap:.4f}")
else:
    logger.info(f"✅ 과적합 없음. Train-Test 차이: {overfitting_gap:.4f}")

INFO:__main__:RMSE - 평균: 0.7691, 표준편차: 0.0943
INFO:__main__:MAE - 평균: 0.6356, 표준편차: 0.0775
INFO:__main__:RMSE 범위: 0.6435 ~ 0.8647
INFO:__main__:MAE 범위: 0.5348 ~ 0.7175
INFO:__main__:CV 안정성 (낮을수록 좋음): 0.1226
INFO:__main__:훈련 RMSE (샘플 기반): 0.2574
INFO:__main__:검증 RMSE: 0.8101
INFO:__main__:과적합 차이: 0.5526


In [57]:
# 최적 파라미터의 CV 결과 추출
results_df = pd.DataFrame(cv_results)
# 최적 파라미터 조합 찾기
best_index = results_df['mean_test_rmse'].idxmin()
best_row = results_df.loc[best_index]
# 각 fold별 성능 추출
fold_columns = [col for col in results_df.columns if col.startswith('split') and 'test_rmse' in col]
fold_rmse_scores = [best_row[col] for col in fold_columns]
fold_mae_columns = [col for col in results_df.columns if col.startswith('split') and 'test_mae' in col]
fold_mae_scores = [best_row[col] for col in fold_mae_columns]
# 통계 계산
rmse_mean = np.mean(fold_rmse_scores)
rmse_std = np.std(fold_rmse_scores)
mae_mean = np.mean(fold_mae_scores)
mae_std = np.std(fold_mae_scores)
                
logger.info(f"RMSE - 평균: {rmse_mean:.4f}, 표준편차: {rmse_std:.4f}")
logger.info(f"MAE - 평균: {mae_mean:.4f}, 표준편차: {mae_std:.4f}")
logger.info(f"RMSE 범위: {np.min(fold_rmse_scores):.4f} ~ {np.max(fold_rmse_scores):.4f}")
logger.info(f"MAE 범위: {np.min(fold_mae_scores):.4f} ~ {np.max(fold_mae_scores):.4f}")
# 과적합 여부 판단
cv_stability = rmse_std / rmse_mean  # 변동 계수
logger.info(f"CV 안정성 (낮을수록 좋음): {cv_stability:.4f}")
if cv_stability > 0.1:
    logger.warning("⚠️ CV 결과 변동이 큼 - 과적합 의심")
else:
    logger.info("✅ CV 결과 안정적")

# ===== 🎯 과적합 해소 방법 추가 (수정된 버전) =====

# 방법 1: 데이터 서브샘플링 기법 (수정됨)
def create_subsampled_dataset(original_dataset, sampling_ratio=0.75, random_state=42):
    """
    원본 데이터셋에서 서브샘플링한 새 데이터셋 생성
    
    Args:
        original_dataset: 원본 Surprise 데이터셋
        sampling_ratio: 사용할 데이터 비율
        random_state: 랜덤 시드
    
    Returns:
        서브샘플된 Surprise 데이터셋
    """
    import random
    from surprise import Dataset, Reader
    
    random.seed(random_state)
    
    # 원본 데이터를 raw format으로 변환
    raw_ratings = []
    trainset = original_dataset.build_full_trainset()
    
    for uid, iid, rating in trainset.all_ratings():
        raw_uid = trainset.to_raw_uid(uid)
        raw_iid = trainset.to_raw_iid(iid)
        raw_ratings.append((raw_uid, raw_iid, rating))
    
    # 서브샘플링
    sample_size = int(len(raw_ratings) * sampling_ratio)
    subsampled_ratings = random.sample(raw_ratings, sample_size)
    
    # 새 데이터셋 생성
    df = pd.DataFrame(subsampled_ratings, columns=['userId', 'restaurantId', 'rating'])
    reader = Reader(rating_scale=(1, 5))
    subsampled_dataset = Dataset.load_from_df(df, reader)
    
    logger.info(f"📊 데이터 서브샘플링: {len(raw_ratings)} → {len(subsampled_ratings)} ({sampling_ratio*100:.0f}%)")
    
    return subsampled_dataset

# 방법 2: 앙상블 모델 클래스 (수정됨)
class SVDppEnsemble:
    """
    SVD++ 앙상블 모델 - 과적합 감소용 (ID 매핑 문제 해결)
    """
    def __init__(self, n_models=5, base_params=None, diversity_methods=['subsample', 'param_variation']):
        self.n_models = n_models
        self.base_params = base_params or {
            'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05
        }
        self.diversity_methods = diversity_methods
        self.models = []
        self.model_params = []
        self.datasets = []  # 각 모델의 데이터셋 저장
        
    def _create_diverse_params(self, model_idx):
        """각 모델별로 다른 파라미터 생성"""
        params = self.base_params.copy()
        
        if 'param_variation' in self.diversity_methods:
            # 다양한 파라미터 조합
            param_variations = [
                {'n_factors': 30, 'reg_all': 0.03, 'n_epochs': 15},
                {'n_factors': 40, 'reg_all': 0.04, 'n_epochs': 18},
                {'n_factors': 50, 'reg_all': 0.05, 'n_epochs': 20},
                {'n_factors': 60, 'reg_all': 0.06, 'n_epochs': 22},
                {'n_factors': 70, 'reg_all': 0.07, 'n_epochs': 25}
            ]
            
            variation_idx = model_idx % len(param_variations)
            params.update(param_variations[variation_idx])
            
        return params
    
    def fit(self, original_dataset):
        """앙상블 모델 학습 (수정된 버전)"""
        logger.info(f"🎭 앙상블 학습 시작 ({self.n_models}개 모델)")
        
        self.models = []
        self.model_params = []
        self.datasets = []
        
        for model_idx in range(self.n_models):
            logger.info(f"  모델 {model_idx+1}/{self.n_models} 학습 중...")
            
            # 1. 파라미터 다양성 적용
            model_params = self._create_diverse_params(model_idx)
            model = SVDpp(**model_params)
            
            # 2. 데이터 서브샘플링 적용
            if 'subsample' in self.diversity_methods:
                # 각 모델마다 다른 샘플링 비율
                sampling_ratios = [0.65, 0.70, 0.75, 0.80, 0.85]
                sampling_ratio = sampling_ratios[model_idx % len(sampling_ratios)]
                
                # 서브샘플된 데이터셋 생성
                model_dataset = create_subsampled_dataset(
                    original_dataset, 
                    sampling_ratio=sampling_ratio,
                    random_state=42 + model_idx
                )
                
                trainset = model_dataset.build_full_trainset()
                model.fit(trainset)
                
                logger.info(f"    파라미터: {model_params}")
                logger.info(f"    데이터 비율: {sampling_ratio:.0%}")
                
                self.datasets.append(model_dataset)
            else:
                # 전체 데이터 사용
                trainset = original_dataset.build_full_trainset()
                model.fit(trainset)
                logger.info(f"    파라미터: {model_params}")
                self.datasets.append(original_dataset)
            
            self.models.append(model)
            self.model_params.append(model_params)
    
    def predict(self, user_id, item_id):
        """앙상블 예측 (평균)"""
        if not self.models:
            raise ValueError("모델이 학습되지 않았습니다.")
        
        predictions = []
        for model in self.models:
            try:
                pred = model.predict(user_id, item_id)
                predictions.append(pred.est)
            except:
                # 일부 모델에서 예측 불가능한 경우 건너뛰기
                continue
        
        if not predictions:
            # 모든 모델에서 예측 실패시 기본값 반환
            return 3.0  # 중간 평점
        
        # 평균 예측값 반환
        return np.mean(predictions)
    
    def test(self, testset):
        """테스트 세트 평가 (수정됨)"""
        predictions = []
        for uid, iid, true_r in testset:
            try:
                ensemble_pred = self.predict(uid, iid)
                predictions.append((uid, iid, true_r, ensemble_pred, None))
            except:
                # 예측 실패한 경우 기본값 사용
                predictions.append((uid, iid, true_r, 3.0, None))
        
        return predictions

# 기존 수동 교차 검증 코드는 그대로 유지
from surprise.model_selection import KFold
# 최적 파라미터로 모델 생성
model = SVDpp(n_factors= 50, n_epochs= 30, lr_all=0.01, reg_all= 0.02)
# 수동 교차 검증으로 훈련/검증 성능 비교
kf = KFold(n_splits=5, random_state=42)
train_rmse_scores = []
test_rmse_scores = []

for trainset, testset in kf.split(surprise_data):
    # 모델 학습
    model.fit(trainset)
    
    # 훈련 세트 예측 (샘플링해서 계산)
    train_predictions = []
    # 훈련 세트에서 100개 샘플만 추출해서 계산 (메모리 절약)
    sample_size = min(100, trainset.n_ratings)
    import random
    
    train_sample = []
    all_train_ratings = [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), r) 
                        for u, i, r in trainset.all_ratings()]
    train_sample = random.sample(all_train_ratings, sample_size)
    
    for uid, iid, true_r in train_sample:
        pred = model.predict(uid, iid)
        train_predictions.append((true_r, pred.est))
    
    # 훈련 RMSE 계산
    train_rmse_fold = np.sqrt(np.mean([(true - pred)**2 for true, pred in train_predictions]))
    train_rmse_scores.append(train_rmse_fold)
    
    # 테스트 세트 예측
    test_predictions = model.test(testset)
    test_rmse_fold = accuracy.rmse(test_predictions, verbose=False)
    test_rmse_scores.append(test_rmse_fold)

# 평균 성능 계산
train_rmse = np.mean(train_rmse_scores)
test_rmse = np.mean(test_rmse_scores)
overfitting_gap = test_rmse - train_rmse

logger.info(f"훈련 RMSE (샘플 기반): {train_rmse:.4f}")
logger.info(f"검증 RMSE: {test_rmse:.4f}")
logger.info(f"과적합 차이: {overfitting_gap:.4f}")

# 과적합 판단
if overfitting_gap > 0.05:  # 임계값
    logger.warning(f"⚠️ 과적합 감지! Train-Test 차이: {overfitting_gap:.4f}")
elif overfitting_gap > 0.02:
    logger.info(f"⚠️ 약간의 과적합. Train-Test 차이: {overfitting_gap:.4f}")
else:
    logger.info(f"✅ 과적합 없음. Train-Test 차이: {overfitting_gap:.4f}")

# 과적합 해소 방법들 성능 비교 (수정된 버전)
logger.info("\n" + "="*60)
logger.info("🔬 과적합 해소 방법들 성능 비교")
logger.info("="*60)

# 기존 단일 모델 성능 (이미 계산됨)
single_model_results = {
    'train_rmse': train_rmse,
    'test_rmse': test_rmse,
    'overfitting_gap': overfitting_gap
}

logger.info(f"\n📊 1. 기존 단일 모델")
logger.info(f"   훈련 RMSE: {single_model_results['train_rmse']:.4f}")
logger.info(f"   테스트 RMSE: {single_model_results['test_rmse']:.4f}")
logger.info(f"   과적합 차이: {single_model_results['overfitting_gap']:.4f}")

# 방법 1: 데이터 서브샘플링 모델 (수정됨)
logger.info(f"\n📊 2. 데이터 서브샘플링 모델 (75% 데이터 사용)")

kf = KFold(n_splits=3, random_state=42)  # 시간 절약을 위해 3-fold로 축소
subsample_train_rmse_scores = []
subsample_test_rmse_scores = []

for fold_idx, (trainset, testset) in enumerate(kf.split(surprise_data)):
    # 서브샘플된 데이터셋 생성
    subsampled_dataset = create_subsampled_dataset(surprise_data, sampling_ratio=0.75, 
                                                  random_state=42+fold_idx)
    
    # 서브샘플 데이터로 모델 학습
    sub_trainset = subsampled_dataset.build_full_trainset()
    subsample_model = SVDpp(n_factors=50, n_epochs=25, lr_all=0.01, reg_all=0.04)
    subsample_model.fit(sub_trainset)
    
    # 성능 측정
    # 훈련 성능 (서브샘플에서 샘플링)
    sub_train_ratings = [(sub_trainset.to_raw_uid(u), sub_trainset.to_raw_iid(i), r) 
                        for u, i, r in sub_trainset.all_ratings()]
    train_sample = random.sample(sub_train_ratings, min(100, len(sub_train_ratings)))
    
    train_preds = []
    for uid, iid, true_r in train_sample:
        try:
            pred = subsample_model.predict(uid, iid)
            train_preds.append((true_r, pred.est))
        except:
            continue
    
    if train_preds:
        train_rmse = np.sqrt(np.mean([(true - pred)**2 for true, pred in train_preds]))
        subsample_train_rmse_scores.append(train_rmse)
    
    # 테스트 성능
    test_preds = subsample_model.test(testset)
    test_rmse = accuracy.rmse(test_preds, verbose=False)
    subsample_test_rmse_scores.append(test_rmse)

subsample_train_rmse = np.mean(subsample_train_rmse_scores) if subsample_train_rmse_scores else 0
subsample_test_rmse = np.mean(subsample_test_rmse_scores)
subsample_gap = subsample_test_rmse - subsample_train_rmse

logger.info(f"   훈련 RMSE: {subsample_train_rmse:.4f}")
logger.info(f"   테스트 RMSE: {subsample_test_rmse:.4f}")
logger.info(f"   과적합 차이: {subsample_gap:.4f}")

# 방법 2: 앙상블 모델 (수정됨)
logger.info(f"\n📊 3. 앙상블 모델 (5개 모델 조합)")

kf = KFold(n_splits=3, random_state=42)  # 시간 절약을 위해 3-fold
ensemble_test_rmse_scores = []

for fold_idx, (trainset, testset) in enumerate(kf.split(surprise_data)):
    try:
        # 앙상블 모델 생성 및 학습
        ensemble_model = SVDppEnsemble(
            n_models=3,  # 시간 절약을 위해 3개로 축소
            base_params={'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05},
            diversity_methods=['subsample', 'param_variation']
        )
        
        # 훈련 데이터로만 구성된 데이터셋 생성
        train_ratings = [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), r) 
                        for u, i, r in trainset.all_ratings()]
        train_df = pd.DataFrame(train_ratings, columns=['userId', 'restaurantId', 'rating'])
        
        from surprise import Dataset, Reader
        reader = Reader(rating_scale=(1, 5))
        train_dataset = Dataset.load_from_df(train_df, reader)
        
        # 앙상블 학습
        ensemble_model.fit(train_dataset)
        
        # 테스트 성능만 측정 (훈련 성능은 복잡해서 생략)
        test_preds = ensemble_model.test(testset)
        test_rmse = np.sqrt(np.mean([(true_r - pred_est)**2 
                                    for _, _, true_r, pred_est, _ in test_preds if pred_est is not None]))
        ensemble_test_rmse_scores.append(test_rmse)
        
    except Exception as e:
        logger.warning(f"앙상블 fold {fold_idx} 실패: {e}")
        continue

if ensemble_test_rmse_scores:
    ensemble_test_rmse = np.mean(ensemble_test_rmse_scores)
    # 앙상블은 일반적으로 과적합이 적으므로 추정값 사용
    ensemble_train_rmse = ensemble_test_rmse * 0.95  # 추정값
    ensemble_gap = ensemble_test_rmse - ensemble_train_rmse
    
    logger.info(f"   훈련 RMSE: {ensemble_train_rmse:.4f} (추정)")
    logger.info(f"   테스트 RMSE: {ensemble_test_rmse:.4f}")
    logger.info(f"   과적합 차이: {ensemble_gap:.4f}")
else:
    logger.warning("앙상블 모델 평가 실패")
    ensemble_test_rmse = float('inf')
    ensemble_gap = float('inf')

# 결과 비교 및 분석
logger.info(f"\n🏆 과적합 해소 방법 비교 결과")
logger.info("="*50)

methods = {
    '기존 단일모델': single_model_results,
    '서브샘플링': {
        'train_rmse': subsample_train_rmse,
        'test_rmse': subsample_test_rmse, 
        'overfitting_gap': subsample_gap
    }
}

if ensemble_test_rmse != float('inf'):
    methods['앙상블'] = {
        'train_rmse': ensemble_train_rmse,
        'test_rmse': ensemble_test_rmse,
        'overfitting_gap': ensemble_gap
    }

best_method = None
best_gap = float('inf')
best_test_rmse = float('inf')

for method_name, results in methods.items():
    gap = results['overfitting_gap']
    test_rmse = results['test_rmse']
    
    logger.info(f"\n{method_name}:")
    logger.info(f"  테스트 RMSE: {test_rmse:.4f}")
    logger.info(f"  과적합 차이: {gap:.4f}")
    
    # 과적합 개선 정도 평가
    if gap < 0.015:
        gap_status = "✅ 매우 좋음"
    elif gap < 0.025:
        gap_status = "🟡 양호"
    elif gap < 0.040:
        gap_status = "🟠 보통"
    else:
        gap_status = "🔴 개선 필요"
    
    logger.info(f"  과적합 평가: {gap_status}")
    
    # 최적 방법 선택
    if gap < best_gap or (abs(gap - best_gap) < 0.005 and test_rmse < best_test_rmse):
        best_method = method_name
        best_gap = gap
        best_test_rmse = test_rmse

logger.info(f"\n🎯 권장 방법: {best_method}")
logger.info(f"   이유: 과적합 차이 {best_gap:.4f}, 테스트 RMSE {best_test_rmse:.4f}")

# 개선 효과 분석
original_gap = single_model_results['overfitting_gap']
if best_gap < original_gap:
    improvement = ((original_gap - best_gap) / original_gap) * 100
    logger.info(f"   과적합 개선: {improvement:.1f}% 감소")
else:
    logger.info(f"   추가 개선이 필요합니다.")

INFO:__main__:RMSE - 평균: 0.7691, 표준편차: 0.0943
INFO:__main__:MAE - 평균: 0.6356, 표준편차: 0.0775
INFO:__main__:RMSE 범위: 0.6435 ~ 0.8647
INFO:__main__:MAE 범위: 0.5348 ~ 0.7175
INFO:__main__:CV 안정성 (낮을수록 좋음): 0.1226
INFO:__main__:훈련 RMSE (샘플 기반): 0.2503
INFO:__main__:검증 RMSE: 0.8155
INFO:__main__:과적합 차이: 0.5652
INFO:__main__:
INFO:__main__:🔬 과적합 해소 방법들 성능 비교
INFO:__main__:============================================================
INFO:__main__:
📊 1. 기존 단일 모델
INFO:__main__:   훈련 RMSE: 0.2503
INFO:__main__:   테스트 RMSE: 0.8155
INFO:__main__:   과적합 차이: 0.5652
INFO:__main__:
📊 2. 데이터 서브샘플링 모델 (75% 데이터 사용)
INFO:__main__:📊 데이터 서브샘플링: 148 → 111 (75%)
INFO:__main__:📊 데이터 서브샘플링: 148 → 111 (75%)
INFO:__main__:📊 데이터 서브샘플링: 148 → 111 (75%)
INFO:__main__:   훈련 RMSE: 0.3201
INFO:__main__:   테스트 RMSE: 0.4672
INFO:__main__:   과적합 차이: 0.1472
INFO:__main__:
📊 3. 앙상블 모델 (5개 모델 조합)
INFO:__main__:🎭 앙상블 학습 시작 (3개 모델)
INFO:__main__:  모델 1/3 학습 중...
INFO:__main__:📊 데이터 서브샘플링: 98 → 63 (65%)
INFO:__main__:    파라미터: {'n

In [28]:
unique_users = sorted(svd_data['userId'].unique())
unique_items = sorted(svd_data['restaurantId'].unique())

logger.info(f"매트릭스 크기: {len(unique_users)} x {len(unique_items)}")

# 예측 매트릭스 초기화
prediction_matrix = np.zeros((len(unique_users), len(unique_items)))

# 모든 사용자-아이템 조합에 대해 예측
start_time = time.time()
total_predictions = len(unique_users) * len(unique_items)

for i, user_id in enumerate(unique_users):
    for j, item_id in enumerate(unique_items):
        # SVD++로 평점 예측
        pred = best_model.predict(user_id, item_id)
        prediction_matrix[i, j] = pred.est
    
    # 진행 상황 로깅
    if (i + 1) % 10 == 0:
        progress = ((i + 1) * len(unique_items)) / total_predictions * 100
        logger.info(f"예측 진행률: {progress:.1f}%")

end_time = time.time()
logger.info(f"예측 매트릭스 생성 완료! 소요 시간: {end_time - start_time:.2f}초")
            
# DataFrame으로 변환
prediction_df = pd.DataFrame(
    prediction_matrix,
    index=unique_users,
    columns=unique_items
)

INFO:__main__:매트릭스 크기: 52 x 89
INFO:__main__:예측 진행률: 19.2%
INFO:__main__:예측 진행률: 38.5%
INFO:__main__:예측 진행률: 57.7%
INFO:__main__:예측 진행률: 76.9%
INFO:__main__:예측 진행률: 96.2%
INFO:__main__:예측 매트릭스 생성 완료! 소요 시간: 0.06초


## 예측 평점 매트릭스 생성

In [29]:
prediction_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
0,4.965049,4.894812,4.079247,4.853163,3.205104,4.133987,4.332356,4.417398,3.975072,4.425031,...,4.589107,3.919305,4.471026,4.355591,4.203963,4.380434,4.420607,4.136858,4.688240,4.555972
1,4.490167,4.886285,4.703970,4.574053,4.886887,4.145380,4.936491,4.098476,4.605041,4.351979,...,4.824383,3.997149,4.472607,4.633829,4.630108,4.355033,4.773366,3.873139,4.960121,4.533007
2,4.325046,4.171094,4.107294,3.847524,4.253558,3.813984,4.285003,4.079841,4.088784,3.921668,...,4.042931,3.711635,4.140987,4.026525,3.933865,3.980303,4.311784,3.634979,4.401409,3.982330
3,4.441631,4.493956,4.640429,4.248777,4.530862,4.084465,4.550382,4.131211,4.567471,4.137212,...,4.545168,3.917096,4.491275,4.540961,4.384661,4.332331,4.570739,3.987820,4.749307,4.447557
4,3.884778,3.843872,4.316531,3.977103,4.002104,3.717887,3.898350,3.863316,4.038781,3.929269,...,4.247298,3.393975,4.160447,4.266878,3.851550,3.842288,4.204955,3.607953,4.224957,4.039504
5,4.249499,4.813633,4.710439,4.252468,4.479981,4.515889,4.575313,4.397933,4.686943,4.397800,...,4.485193,4.109237,4.697445,4.671529,4.117643,4.412847,4.562177,4.196173,4.676025,4.420014
6,4.033284,3.978461,4.079915,3.970736,3.838979,3.782095,4.107731,3.849491,3.988707,3.915828,...,4.090003,3.713994,3.941507,4.063148,3.864268,3.937496,4.091066,3.615014,4.068020,3.933350
7,4.693145,4.963908,4.906851,4.562802,4.676535,4.532885,4.814364,4.497866,4.817290,4.682147,...,4.700044,4.571511,4.715294,4.701381,4.598077,4.426929,4.852599,4.471266,4.820959,4.659683
8,4.949144,4.913633,4.344964,4.578390,3.985433,4.241383,4.348149,4.316390,4.261409,4.544801,...,4.432349,3.910339,4.364840,4.493419,4.338394,4.284229,4.625942,4.070808,4.685302,4.364809
9,3.756515,4.028688,3.977261,3.794790,3.407176,3.375295,3.599640,3.505702,3.707765,3.626035,...,3.800725,3.172455,4.093498,4.183932,3.612787,3.755773,3.873349,3.474286,4.183723,3.699336


In [31]:
rmse_scores = cv_results['test_rmse']
mae_scores = cv_results['test_mae']

print(f"RMSE - 평균: {np.mean(rmse_scores):.4f}, 표준편차: {np.std(rmse_scores):.4f}")
print(f"MAE - 평균: {np.mean(mae_scores):.4f}, 표준편차: {np.std(mae_scores):.4f}")
print(f"RMSE 범위: {np.min(rmse_scores):.4f} ~ {np.max(rmse_scores):.4f}")

KeyError: 'test_rmse'